In [1]:
import time

from selenium.webdriver import Chrome, Firefox
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.common.exceptions import NoSuchElementException

In [2]:
import pandas as pd
import re

In [3]:
def scrollDown(driver, n_scroll):
    body = driver.find_element_by_tag_name("body")
    while n_scroll >= 0:
        body.send_keys(Keys.PAGE_DOWN)
        n_scroll -= 1
    return driver

driver = Firefox(executable_path='C:/Program Files/Mozilla Firefox/geckodriver-v0.26.0-win64/geckodriver.exe')
#driver = webdriver.firefox(executable_path='C:/Program Files/Mozilla Firefox/geckodriver-v0.26.0-win64/geckodriver.exe')

url = 'https://www.sephora.sg'
driver.get(url)


In [4]:
# initiate empty dataframe
df = pd.DataFrame(columns=['Label', 'URL'])
print(df)

Empty DataFrame
Columns: [Label, URL]
Index: []


In [5]:
# step 1
tickers = ['cleanser-and-exfoliator', 'toner', 'moisturiser', 'masks-and-treatments',
           'suncare']

for ticker in tickers:
    url = 'https://www.sephora.sg/categories/skincare/' + ticker + '?view=120'
    driver.get(url)

#     xpath = '/html/body/div[9]/div/div/div[1]/div/div/button'
#     btn = driver.find_element_by_xpath(xpath)
#     btn.click()
#     time.sleep(3)

    browser = scrollDown(driver, 10)
    time.sleep(1)

    browser = scrollDown(driver, 10)
    time.sleep(1)

    browser = scrollDown(driver, 10)
    time.sleep(1)

    browser = scrollDown(driver, 10)

    # Where links for href is found:
    element = driver.find_elements_by_class_name('product-card-image-container')

    subpageURL = []
    for a in element:
        subURL = a.get_attribute('href')
        subpageURL.append(subURL)

    # transform into a data frame
    dic = {'Label': ticker, 'URL': subpageURL}
    df = df.append(pd.DataFrame(dic), ignore_index = True)




In [6]:
# add columns
df2 = pd.DataFrame(columns=['brand', 'name', 'price', 'rank', 'skin_type', 'ingredients'])
df = pd.concat([df, df2], axis = 1)

In [7]:
# step 2
for i in range(len(df)+1):
    url = df.URL[i]
    driver.get(url)
    time.sleep(5)

    # brand, name, price    
    try:
        df.brand[i] = driver.find_element_by_class_name('product-brand').text
        df.name[i] = driver.find_element_by_class_name('product-heading').text
        df.price[i] = driver.find_element_by_xpath('/html/body/div[2]/div/div/div[2]/main/div/div/div[2]/div[2]/div[1]/div[2]/div[1]/p/span').text
                
    except:
        pass

#     browser = scrollDown(driver, 1)
#     time.sleep(5)
#     browser = scrollDown(driver, 1)
#     time.sleep(5)

    # skin_type            
    try:
        df.skin_type[i] = driver.find_element_by_class_name('product-filter-type-value').text 
        
    except:
         pass
        
    # ingredients
    try:
        xpath = '.product-ingredients > div:nth-child(3) > div:nth-child(2) > button:nth-child(1)'
        btn = driver.find_element_by_css_selector(xpath)               
        btn.click()
        time.sleep(2)
        df.ingredients[i] = driver.find_element_by_class_name('product-ingredients-values').text
        
    except NoSuchElementException:
        df.ingredients[i] = 'No Info'

        
    # rank
    try:
        rank = driver.find_element_by_class_name('rateit-range').get_attribute('aria-valuenow')
        df['rank'][i] = rank
        
    except NoSuchElementException:
        df['rank'][i] = 0
 
    # Just for Verbose
    print(i)    




0
1
2


KeyboardInterrupt: 

In [8]:
df.to_csv('sephora_SG_FIREFOX_dataset.csv', encoding = 'utf-8-sig', index = False)

In [8]:
df.head(10)

,Label,URL,brand,name,price,rank,skin_type,ingredients
0,cleanser-and-exfoliator,https://www.sephora.sg/products/fresh-soy-face...,FRESH,Soy Face Cleanser,$113.00,4.5,"Skin Type: Combination, Dry, Normal, Oily, Sen...","Aqua (Water), Coco-Glucoside, Glycerin, Butyle..."
1,cleanser-and-exfoliator,https://www.sephora.sg/products/indie-lee-brig...,INDIE LEE,Brightening Cleanser,$50.00,4.5,"Skin Type: Combination, Dry, Normal, Oily, Sen...","Water (Aqua/Eau), Decyl Glucoside (Veg. Oils &..."
2,cleanser-and-exfoliator,https://www.sephora.sg/products/tarte-sea-deep...,TARTE,Sea Deep Dive Makeup Removing Gel Cleanser,$40.00,4.5,"Skin Type: Combination, Dry, Normal, Oily, Sen...","Water/Aqua/Eau, cocamidopropyl betaine, disodi..."
3,cleanser-and-exfoliator,https://www.sephora.sg/products/fresh-rose-cle...,NaN,NaN,NaN,NaN,NaN,NaN
4,cleanser-and-exfoliator,https://www.sephora.sg/products/first-aid-beau...,NaN,NaN,NaN,NaN,NaN,NaN
5,cleanser-and-exfoliator,https://www.sephora.sg/products/dr-roebucks-no...,NaN,NaN,NaN,NaN,NaN,NaN
6,cleanser-and-exfoliator,https://www.sephora.sg/products/it-cosmetics-c...,NaN,NaN,NaN,NaN,NaN,NaN
7,cleanser-and-exfoliator,https://www.sephora.sg/products/votary-cleansi...,NaN,NaN,NaN,NaN,NaN,NaN
8,cleanser-and-exfoliator,https://www.sephora.sg/products/tata-harper-no...,NaN,NaN,NaN,NaN,NaN,NaN
9,cleanser-and-exfoliator,https://www.sephora.sg/products/fresh-sugar-st...,NaN,NaN,NaN,NaN,NaN,NaN
